In [9]:
# Import required libraries
import pandas as pd

# Load the Excel file
file_path = 'shihara_dataset.xlsx'  # Update this if the file path is different
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe to check the data
df.head()


,Date,Particulars,Payments,Receipts,Balance
0,2022-12-27,IB CEFT CHGS K A Kumaranayek,30,NaN,8449.35
1,2022-12-27,PURCHASE UNIVERSITY GRANTS C,50,NaN,8399.35
2,2022-12-28,PURCHASE KEELLS SUPER - MORA,5000,NaN,3399.35
3,2022-12-30,INTEREST,NaN,41.57,3440.92
4,2023-01-04,NOV RENT,3000,75000.00,78440.92


In [11]:
# Inspect the columns to ensure they exist
print(df.columns)

# If necessary, clean the data (e.g., handling missing values)
df['Particulars'] = df['Particulars'].fillna('')

# Optionally, you can include payment, receipt, and balance as metadata in ChromaDB


Index(['Date', 'Particulars', 'Payments', 'Receipts', 'Balance'], dtype='object')


In [12]:
# Initialize ChromaDB client
from chromadb import Client

# Connect to the ChromaDB client
client = Client()

# Create or load the collection (named 'banking_transactions')
collection_name = 'banking_transactions'
collection = client.get_or_create_collection(collection_name)


In [14]:
# Import SentenceTransformer
from sentence_transformers import SentenceTransformer

# Load pre-trained transformer model for sentence embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings for all the transaction particulars
embeddings = model.encode(df['Particulars'].tolist())

# Optionally, view the size of the embeddings
print(f"Generated embeddings of shape: {embeddings.shape}")


Generated embeddings of shape: (557, 384)


In [17]:
# Insert embeddings and their metadata into ChromaDB collection
for idx, emb in enumerate(embeddings):
    # Create a metadata dictionary for each transaction
    metadata = {
        "Particulars": df['Particulars'].iloc[idx],
        "Payments": df['Payments'].iloc[idx],  # Or handle missing values as needed
        "Receipts": df['Receipts'].iloc[idx],
        "Balance": df['Balance'].iloc[idx]
    }
    
    # Add the embedding and metadata into the collection
    collection.add(id=str(idx), embeddings=emb.tolist(), metadata=metadata)

print(f"Inserted {len(embeddings)} embeddings into the collection.")


TypeError: Collection.add() got an unexpected keyword argument 'id'